# FAISS AI-Search für PubMed
Mithilfe dieses Scripts kann man Artikel aus der PubMed Datenbank auf Relevanz für eine vorgegebene These mithilfe von MSR BiomedBERT.

© Lino Brendler, 2025


In [1]:
!pip install pubmed_parser

In [2]:
!pip install -U langchain-community

In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 55.8 MB/s eta 0:00:00


In [8]:
import xml.etree.ElementTree as ET
import requests

# Import
from transformers import AutoTokenizer, AutoModel, pipeline

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain

########################################
# 1️⃣ BERT-basiertes Modell für Embeddings
########################################

# Modell: MSR BiomedBERT (speziell für biomed. Texte)
# Quelle: https://huggingface.co/microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext
embedding_model_name = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"

# Wir erzeugen ein Embedding-Objekt.
embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_name
)

########################################
# 2️⃣ XML-Parser: Titel, Abstract, Autoren
########################################
def parse_minimal_pubmed_xml(xml_str):
    try:
        root = ET.fromstring(xml_str)
    except ET.ParseError:
        return {
            'title': '',
            'abstract': '',
            'authors': []
        }

    article_el = root.find('.//Article')
    if article_el is None:
        return {
            'title': '',
            'abstract': '',
            'authors': []
        }

    title = article_el.findtext('ArticleTitle', default='')

    abstract_texts = article_el.findall('.//AbstractText')
    full_abstract = ' '.join(
        t.text for t in abstract_texts if t is not None and t.text
    )

    authors_el = article_el.find('.//AuthorList')
    authors = []
    if authors_el is not None:
        for author in authors_el.findall('Author'):
            last_name = author.findtext('LastName', '').strip()
            fore_name = author.findtext('ForeName', '').strip()
            name = (fore_name + ' ' + last_name).strip()
            if name:
                authors.append(name)

    return {
        'title': title,
        'abstract': full_abstract,
        'authors': authors
    }

########################################
# 3️⃣ PubMed-Artikel abrufen
########################################
def get_pubmed_articles(topic, max_results=100):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        "db": "pubmed",
        "term": topic,  # Nur nach thematisch relevanten Artikeln suchen
        "retmode": "json",
        "retmax": max_results
    }
    response = requests.get(base_url, params=params)
    ids = response.json().get("esearchresult", {}).get("idlist", [])

    articles = []
    for pmid in ids:
        article_data = get_pubmed_abstract(pmid)
        if article_data and article_data.get('abstract'):
            articles.append(article_data)
    return articles


def get_pubmed_abstract(pmid):
    fetch_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id={pmid}&retmode=xml"
    response = requests.get(fetch_url)
    if response.status_code != 200:
        return None

    parsed = parse_minimal_pubmed_xml(response.text)
    if not parsed.get('title') and not parsed.get('abstract'):
        return None

    authors_str = ", ".join(parsed['authors'])

    return {
        "pmid": pmid,
        "title": parsed['title'],
        "abstract": parsed['abstract'],
        "authors": authors_str
    }

########################################
# 4️⃣ Erstelle FAISS-Vektordatenbank
########################################
def build_faiss_db(articles):
    texts = [a["abstract"] for a in articles]
    metadata = [
        {
            "pmid": a["pmid"],
            "title": a["title"],
            "authors": a.get("authors", "")
        }
        for a in articles
    ]
    faiss_db = FAISS.from_texts(texts, embedding_model, metadatas=metadata)
    return faiss_db

########################################
# 5️⃣ Semantische Suche in der FAISS-Datenbank
########################################
def search_pubmed(thesis, db, k=10):
    similar = db.similarity_search(thesis, k=k)
    results = []
    for doc in similar:
        results.append(
            f"Title: {doc.metadata['title']}\n" +
            f"Authors: {doc.metadata['authors']}\n" +
            f"PMID: {doc.metadata['pmid']}\n" +
            f"Abstract: {doc.page_content}\n"
        )
    return "\n\n".join(results)

########################################
# 7️⃣ Aufruf
########################################
topic = "(machine learning[Title/Abstract]) AND (cancer[Title/Abstract])" # hier PubMed Search Term eingeben
thesis = "Machine learning improves cancer diagnosis" # hier These eingeben

print(f"📄 Suche nach Artikeln zum Thema '{topic}'...") # sucht nach Artikeln zu dem Search Term
articles = get_pubmed_articles(topic, max_results=1000) # hier Maximale Ergebnis Zahl einsetzen, für genaue Analyse ab 10_000 (dauert dann aber entsprechend länger)
print(f"📄 {len(articles)} Artikel gefunden. Erstelle FAISS-Datenbank...") # erstellung der FAISS Vector-Search Datenbank
faiss_db = build_faiss_db(articles)

print("🔍 Suche nach relevanten Publikationen zur These...") # heraussuchen der relevanten artikel mithilfe von FAISS
search_results = search_pubmed(thesis, faiss_db, k=10) # k-Wert anpassen, so höher desto ungenauer die Auswahl, desto mehr Artikel bleiben übrig, kleiner k-Wert bedeutet genauere Ergebnisse, dafür weniger
print(search_results)


📄 Suche nach Artikeln zum Thema '(machine learning[Title/Abstract]) AND (cancer[Title/Abstract])'...
📄 229 Artikel gefunden. Erstelle FAISS-Datenbank...
🔍 Suche nach relevanten Publikationen zur These...
Title: Continuing Medical Education Questions: August 2024.
Authors: Shifa Umar
PMID: 39724582
Abstract: Article Title: Machine Learning Models for Pancreatic Cancer Risk Prediction Using Electronic Health Record Data-A Systematic Review and Assessment.


Title: An Adaptive Dendritic Neural Model for Lung Cancer Prediction.
Authors: Umair Arif, Chunxia Zhang, Muhammad Waqas Chaudhary, Sajid Hussain
PMID: 40026264
Abstract: Lung cancer is a leading cause of cancer-related deaths, often diagnosed late due to its aggressive nature. This study presents a novel Adaptive Dendritic Neural Model (ADNM) to enhance diagnostic accuracy in high-dimensional healthcare data. Utilizing hyperparameter optimization and activation mechanisms, ADNM improves scalability and feature selection for multi-cla

Zuerst folgende Vorlage kopieren (nach ChatGPT):

```
Hier ist eine These:
[These]
Ich habe mehrere Artikel herausgesucht, mit deren Hilfe möchte ich die These stärken. Nenne für jeden artikel ob er relevant ist oder nicht in folgendem Format:

---
Titel: [Titel]
PMID: [PMID]
Relevanz: [Relevanz von 1 (komplett irrelevant) bis 10 (100% relevant)]
Begründung der Relevanz: [Begründung]
---
[Nächste Artikel im selben Format verarbeiten!]
---

Hier sind die Artikel:
[Artikel]

```

Schreiben sie in `[These]` ihre These mit der sie gesucht haben (also von oben kopieren). In `[Artikel]` müssen sie alle Artikel im Format von oben reinkopieren.
Jetzt müssen sie sich nur noch die Ergebnisse von ChatGPT durchschauen.